# Analysis of Traffic Collision Patterns: Time, Weather and Road Factors

### Step 1: CollisionByWeather Data Loading from CSV to python dataframe: 

In [3]:
#import required libraries
import pandas as pd
import csv

import mysql.connector
from mysql.connector import errorcode
#!pip install tabulate
#from tabulate import tabulate

import matplotlib.pyplot as plt

#!pip install SQLAlchemy pymysql
from sqlalchemy import create_engine

import re

In [5]:
# CollisionByWeather
url_CollisionByWeather = "https://raw.githubusercontent.com/anithamonica/DATA604_TrafficIncidents/main/datasets/CollisionByWeather.csv"
colNamesColsnByWeather = ['Month', 'Year', 'RoadSurface', 'Weather',  
              'NoofCollisions', 'NoofInjured', 'NoofFatalities']
dfByWeather = pd.read_csv(url_CollisionByWeather, skiprows=3, names = colNamesColsnByWeather)
display(dfByWeather.head())

,Month,Year,RoadSurface,Weather,NoofCollisions,NoofInjured,NoofFatalities
0,January,2010,"Dry, normal",Clear and sunny,2954,4102,50
1,January,2010,"Dry, normal","Overcast, cloudy but no precipitation",352,485,9
2,January,2010,"Dry, normal",Raining,9,10,0
3,January,2010,"Dry, normal","Snowing, not including drifting snow",6,6,1
4,January,2010,"Dry, normal","Freezing rain, sleet, hail",1,1,0


In [7]:
print("----------------------------------------------------------------------------")
print("\033[1m"+"Data Analysis of CollisionByWeather"+"\033[0m")
print("----------------------------------------------------------------------------")

#display shape, columns, and data types
print("1.\tShape of the CollisionByWeather dataset:", dfByWeather.shape)
print("2.\tNumber of records or rows of the DataFrame:", dfByWeather.shape[0])
print("3.\tColumns and Data types of each column:\n", dfByWeather.dtypes)
dfByWeatherLength = len(dfByWeather)

----------------------------------------------------------------------------
Data Analysis of CollisionByWeather
----------------------------------------------------------------------------
1.	Shape of the CollisionByWeather dataset: (15444, 7)
2.	Number of records or rows of the DataFrame: 15444
3.	Columns and Data types of each column:
 Month             object
Year               int64
RoadSurface       object
Weather           object
NoofCollisions     int64
NoofInjured        int64
NoofFatalities     int64
dtype: object


In [9]:
#Inspecting data for CollisionByWeather Dataframe
missingDataSum = dfByWeather.isna().sum()
missingDataPercentage = (dfByWeather.isnull().mean() * 100).round(2)
missingData = pd.DataFrame({
    "Missing Count": missingDataSum,
    "Missing Percentage": missingDataPercentage
})

pd.options.display.float_format = '{:.2f}'.format
print("\n\033[1m"+"Missing Count per column:"+"\033[0m")
#print(tabulate(missingData, headers='keys', tablefmt='fancy_grid'))
display(missingData)

print("Unique values")
print("----------------------------------------------------------------------------")
print(dfByWeather['Month'].unique())
print(dfByWeather['Year'].unique())
print(dfByWeather['RoadSurface'].unique())
print(dfByWeather['Weather'].unique())


Missing Count per column:


,Missing Count,Missing Percentage
Month,0,0.00
Year,0,0.00
RoadSurface,0,0.00
Weather,0,0.00
NoofCollisions,0,0.00
NoofInjured,0,0.00
NoofFatalities,0,0.00


Unique values
----------------------------------------------------------------------------
['January' 'February' 'March' 'April' 'May' 'June' 'July' 'August'
 'September' 'October' 'November' 'December' 'Unknown']
[2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
['Dry, normal' 'Wet' 'Snow (fresh, loose snow)' 'Slush, wet snow' 'Icy'
 'Sand/gravel/dirt' 'Muddy' 'Oil' 'Flooded' 'Other' 'Unknown']
['Clear and sunny' 'Overcast, cloudy but no precipitation' 'Raining'
 'Snowing, not including drifting snow' 'Freezing rain, sleet, hail'
 'Visibility limitation' 'Strong wind' 'Other' 'Unknown']


### Step 2: CollisionByWeather Data Cleaning and preprocessing 

In [15]:
print(len(dfByWeather[(dfByWeather['NoofCollisions'] == 0
      & (dfByWeather['NoofInjured'] != 0 |
         (dfByWeather['NoofFatalities'] != 0)))]))

7558


In [17]:
#Drop invalid records
#----------------------------------------------------------------------------

#Below are the records to be dropped
print("Below are the records to be dropped")
print("----------------------------------------------------------------------------")
print("Count of records with NoofCollisions = 0 is:", len(dfByWeather[dfByWeather['NoofCollisions'] == 0]))

#Drop the records with collisionCount = 0 => invalid records
dfByWeatherLength = len(dfByWeather)
dfByWeather = dfByWeather[dfByWeather['NoofCollisions'] != 0]
newdfByWeatherLen = len(dfByWeather)
deleteCount = dfByWeatherLength - newdfByWeatherLen
print(f"Deleted {deleteCount} records where NoofCollisions != 0")
print("Number of records after deleting with collisions count != 0 criteria is:", newdfByWeatherLen)
#display(dfByWeather[dfByWeather['NoofCollisions'] == 0])

Below are the records to be dropped
----------------------------------------------------------------------------
Count of records with NoofCollisions = 0 is: 7558
Deleted 7558 records where NoofCollisions != 0
Number of records after deleting with collisions count != 0 criteria is: 7886


In [23]:
#Impute missing values
#----------------------------------------------------------------------------

ambiguousValues = [
    'Choice is other than the preceding values',
    'Unknown',
    'Jurisdiction does not provide this data element'
]

#Below are the records to be imputed
print("\nBelow are the records to be imputed")
print("----------------------------------------------------------------------------")
print("Count of records with Month = 'Unknown' is :", len(dfByWeather[dfByWeather['Month'] == 'Unknown']))
print("Count of records with ambiguous RoadSurface values are:", dfByWeather['RoadSurface'].isin(ambiguousValues).sum())
print("Count of records with ambiguous Weather values are:", dfByWeather['Weather'].isin(ambiguousValues).sum())


#Impute Month with most frequent value of Month
modeMonth = dfByWeather['Month'].mode()[0]
dfByWeather['Month'] = dfByWeather['Month'].apply(lambda x: modeMonth if x == 'Unknown' else x)
print("\nImputed Month with it's mode: ", modeMonth)

#Impute RoadClass with most frequent value of RoadClass
modeRoadSurface = dfByWeather['RoadSurface'].mode()[0]
dfByWeather['RoadSurface'] = dfByWeather['RoadSurface'].apply(lambda x: modeRoadSurface if x == 'Unknown' else x)
print("\nImputed RoadSurface with it's mode: ", modeRoadSurface)

#Impute Pedestrains with most frequent value of Pedestrains
modeWeather = dfByWeather['Weather'].mode()[0]
dfByWeather['Weather'] = dfByWeather['Weather'].apply(lambda x: modeWeather if x == 'Unknown' else x)
print("\nImputed Weather with it's mode: ", modeWeather)      


#print("Count of records with Month = 'Unknown' is :", len(dfByWeather[dfByWeather['Month'] == 'Unknown']))
#print("Count of records with ambiguous RoadSurface values are:", dfByWeather['RoadSurface'].isin(ambiguousValues).sum())
print("Count of records with ambiguous Weather values are:", dfByWeather['Weather'].isin(ambiguousValues).sum())


#print(dfByWeather['Month'].unique())
#print(dfByWeather['Year'].unique())
#print(dfByWeather['RoadSurface'].unique())
#print(dfByWeather['Weather'].unique())


Below are the records to be imputed
----------------------------------------------------------------------------
Count of records with Month = 'Unknown' is : 38
Count of records with ambiguous RoadSurface values are: 795
Count of records with ambiguous Weather values are: 870

Imputed Month with it's mode:  January

Imputed RoadSurface with it's mode:  Wet

Imputed Weather with it's mode:  Clear and sunny
Count of records with Month = 'Unknown' is : 0
Count of records with ambiguous RoadSurface values are: 0
Count of records with ambiguous Weather values are: 0
['January' 'February' 'March' 'April' 'May' 'June' 'July' 'August'
 'September' 'October' 'November' 'December']
[2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
['Dry, normal' 'Wet' 'Snow (fresh, loose snow)' 'Slush, wet snow' 'Icy'
 'Sand/gravel/dirt' 'Muddy' 'Other' 'Oil' 'Flooded']
['Clear and sunny' 'Overcast, cloudy but no precipitation' 'Raining'
 'Snowing, not including drifting snow' 'Freezing rain, sleet,

In [25]:
#Introduce new Column : Season
#----------------------------------------------------------------------------

# Mapping from month name to season
monthToSeason = {
    'December': 'Winter', 'January': 'Winter', 'February': 'Winter',
    'March': 'Spring', 'April': 'Spring', 'May': 'Spring',
    'June': 'Summer', 'July': 'Summer', 'August': 'Summer',
    'September': 'Fall', 'October': 'Fall', 'November': 'Fall'
}
dfByWeather['Season'] = dfByWeather['Month'].map(monthToSeason)
print("Column 'Season' is inserted")
display(dfByWeather.head(5))

Column 'Season' is inserted


,Month,Year,RoadSurface,Weather,NoofCollisions,NoofInjured,NoofFatalities,Season
0,January,2010,"Dry, normal",Clear and sunny,2954,4102,50,Winter
1,January,2010,"Dry, normal","Overcast, cloudy but no precipitation",352,485,9,Winter
2,January,2010,"Dry, normal",Raining,9,10,0,Winter
3,January,2010,"Dry, normal","Snowing, not including drifting snow",6,6,1,Winter
4,January,2010,"Dry, normal","Freezing rain, sleet, hail",1,1,0,Winter


### Step 3: Import cleaned CollisionByWeather dataset to MySQL database

In [ ]:
USER = "student"
DB   = "student"
    
# attempt a connection
myconnection = mysql.connector.connect(user=USER, 
                                       password='Bi3KSjqgrNOOL',
                                       host='127.0.0.1', 
                                       port=3306,
                                       database=DB,
                                       allow_local_infile=True)
myconnection

In [ ]:
# Create table CollisionByWeather
queryCreate = '''CREATE TABLE student.CollisionByWeather(
    CollisionMonth varchar(15) NOT NULL,
    CollisionYear int, 
    RoadSurface varchar(25),
    Weather varchar(25),
    NoofCollisions int,
    NoofInjured int,
    NoofFatalities int,
    Season varchar(10));'''

createCursor = myconnection.cursor()
try:
    createCursor.execute(queryCreate)
    print("CollisionByWeather table created successfully!")
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("Error! Table CollisionByWeather is already created.")
    else:
        print(err.msg)

createCursor.close()

In [ ]:
# Insert into table CollisionByWeather
insertCursor = myconnection.cursor()

try:
    myconnection.start_transaction()

    for i, currentRow in dfByWeather.iterrows():
        insertCommand = "INSERT INTO `CollisionByWeather` VALUES (" + "%s," * (len(currentRow) - 1) + "%s)"
        insertCursor.execute(insertCommand, tuple(currentRow))

    myconnection.commit()
    print("CollisionByWeather: Inserted records successfully")

except Exception as e:
    myconnection.rollback()
    print("Transaction rolled back due to error:", e)

finally:
    insertCursor.close()

In [ ]:
read_cursor = myconnection.cursor(buffered=True, dictionary=True)
query_string = ("SELECT COUNT(*) FROM CollisionByWeather;")
read_cursor.execute(query_string)

for (library_value) in read_cursor:
    print(library_value)
read_cursor.close()
myconnection.close()

### Step 4: CollisionByWeather data analysis and visualization

 - Which season impact collision frequency and severity? 
 - Is there any relationship between environmental factors like snow, rain, or fog and the number of injuries or fatalities?



In [ ]:
#Which season impact collision frequency and severity? 
queryMonthly = '''SELECT 
    Season,
    SUM(NoofCollisions) AS TotalCollisions,
    ROUND(SUM(NoofInjured) * 1.0 / SUM(NoofCollisions), 2) AS AvgInjuriesPerCollision,
    ROUND(SUM(NoofFatalities) * 1.0 / SUM(NoofCollisions), 4) AS FatalityRate
FROM 
    student.CollisionByWeather
GROUP BY 
    Season
ORDER BY 
    TotalCollisions DESC;'''
result = pd.read_sql_query(queryMonthly, engine)
display(result)

# Pivot so each SpeedLimit shows Urban and Rural side-by-side
pivot_df = result.pivot(index='SpeedValue', columns='RoadClass', values='InjurySeverity')

# Plot
pivot_df.plot(kind='bar', figsize=(14, 6))
plt.title('Injury Severity by Speed Limit and Road Class')
plt.ylabel('Injury Severity')
plt.xticks(rotation=45)
plt.ylim(bottom=1.0) 
plt.legend(title='Road Class', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
#Is there any relationship between environmental factors like snow, rain, or fog and the number of injuries or fatalities?

queryMonthly = '''SELECT 
    Weather,
    SUM(NoofCollisions) AS TotalCollisions,
    ROUND(SUM(NoofInjured) * 1.0 / SUM(NoofCollisions), 2) AS AvgInjuriesPerCollision,
    ROUND(SUM(NoofFatalities) * 1.0 / SUM(NoofCollisions), 4) AS FatalityRate
FROM 
    student.WeatherCollisions   -- replace with your actual table name
WHERE 
    NoofCollisions > 0
GROUP BY 
    Weather
ORDER BY 
    FatalityRate DESC;'''
result = pd.read_sql_query(queryMonthly, engine)
display(result)

# Pivot so each SpeedLimit shows Urban and Rural side-by-side
pivot_df = result.pivot(index='SpeedValue', columns='RoadClass', values='InjurySeverity')

# Plot
pivot_df.plot(kind='bar', figsize=(14, 6))
plt.title('Injury Severity by Speed Limit and Road Class')
plt.ylabel('Injury Severity')
plt.xticks(rotation=45)
plt.ylim(bottom=1.0) 
plt.legend(title='Road Class', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
engine.dispose()